### submet

### Usage
This is a package to compute the distance between equi-dimensional subspaces.  Each metric used to compute the distance between subspaces is dependent on the singular values of the inner product of the two subspaces.  Given two subspaces, $X$, and $Y$, each of dimension $p$, we can compute the singular values $\sigma_{1}... \sigm_{p}$.  This package implements a variety of metrics, including:

 * Asimov
 
 $\begin{align}
 d(X,Y) = \Big(\sum^{p}_{i=1}\theta_{i}^{2})
 \end{align}$
 
 * Binet-Cauchy
 * Chordal
 * Fubini-Study
 * Grassmann
 * Martin
 * Procrustes
 * Projection
 * Spectral

In [30]:
import numpy as np
from niio import loaded, write

from fragmenter import RegionExtractor as re

%matplotlib inline
import matplotlib.pyplot as plt

In [8]:
home_dir = '/Users/kristianeschenburg/Desktop/'
data_dir = '%sData/' % (home_dir)
conn_dir = '%sReorient/Pairwise/' % (home_dir)

label_file = '%sLabels/Desikan/L.100.aparc.32k_fs_LR.label.gii' % (data_dir)
R = re.Extractor(label_file)
index_map = R.map_regions()

In [10]:
sreg = 'inferiorparietal'
treg = 'precentral'

In [90]:
conn_1_file = '%s%s/285345.L.%s.2.%s.Evecs.Full.Signed.Reorient.func.gii' % (conn_dir, sreg, sreg, treg)
conn_1 = loaded.load(conn_1_file)[index_map[sreg], :2]

conn_2_file = '%s%s/285446.L.%s.2.%s.Evecs.Full.Signed.Reorient.func.gii' % (conn_dir, sreg, sreg, treg)
conn_2 = loaded.load(conn_2_file)[index_map[sreg], :2]

In [115]:
def grassmann(theta):
    
    """
    Compute Grassman distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """

    return np.sqrt((theta**2).sum())

def asimov(theta):
    
    """
    Compute Asimov distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """

    return theta[-1]

def binetcauchy(theta):
    
    """
    Compute Binet-Cauchy distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """
    
    return np.sqrt((1 - np.prod(np.cos(theta)**2)))

def chordal(theta):
    
    """
    Compute Chordal distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """
    
    return np.sqrt((np.sin(theta)**2).sum())

def fubinistudy(theta):
    
    """
    Compute Fubini-Study distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """
    
    return np.arccos(np.prod(np.cos(theta)))

def martin(theta):
    
    """
    Compute Martin distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """
    
    return np.sqrt(np.log(np.prod((1/(np.cos(theta)**2)))))

def procrustes(theta):
    
    """
    Compute Procrustes distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """
    
    return 2*np.sqrt((np.sin(theta/2)**2).sum())

def projection(theta):
    
    """
    Compute Projection distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """
    
    return np.sin(theta[-1])

def spectral(theta):
    
    """
    Compute Projection distance between two equi-dimensional subspaces.
    
    Parameters:
    - - - - -
    theta: float, array
        arccos(singular values) of inner product of two equi-dimensional subspaces
    """
    
    return 2*np.sin(theta[-1]/2)

    
class SubspaceDistance(object):
    
    """
    Class to compute a variety of distance between subspaces of equal dimension.
    """
    
    def __init__(self, metric='Grassmann'):
        
        """
        Parameters:
        - - - - -
        metric: string
            metric / distance to use
        """

        assert metric in ['Asimov', 'BinetCauchy', 'Chordal', 'FubiniStudy', 
                          'Grassmann', 'Martin', 'Procrustes', 'Projection',
                          'Spectral']
        
        self.metric = metric
        self.distance_map = {'Asimov': metric.asimov,
                             'BinetCauchy': metric.binetcauchy,
                             'Chordal': metric.chordal,
                             'FubiniStudy': metric.fubinistudy,
                             'Grassmann': metric.grassmann,
                             'Martin': metric.martin,
                             'Procrustes': metric.procrustes,
                             'Projection': metric.projection,
                             'Spectral': metric.spectral}
    
    
    def fit(self, X, Y):
        
        """
        Fit subspace distance between two equi-dimensional subspaces.
        
        Parameters:
        - - - - -
        X, Y: float, array
            two equi-dimensional subspaces
        """
        
        p = np.min([X.shape[1], Y.shape[1]])
        [q1, r1] = np.linalg.qr(X)
        [q2, r2] = np.linalg.qr(Y)
        
        S = q1.T.dot(q2)
        [u, s, v] = np.linalg.svd(S, full_matrices=False)
        
        theta = np.arccos(s)
        
        self.x_ = q1.dot(u)
        self.y_ = q2.dot(v)
        
        print('Computing %s distance.' % (self.metric))
        self.distance_ = self.distance_map[self.metric](theta)
        

In [120]:
for metric in ['Asimov', 'BinetCauchy', 'Chordal', 'FubiniStudy',  'Grassmann', 'Martin', 'Procrustes', 'Projection', 'Spectral']:
    SD = SubspaceDistance(metric=metric)
    SD.fit(conn_1, conn_2)
    print(SD.distance_)

Computing Asimov distance.
0.47730547
Computing BinetCauchy distance.
0.5095130031375166
Computing Chordal distance.
0.5221057
Computing FubiniStudy distance.
0.5346186
Computing Grassmann distance.
0.5391515
Computing Martin distance.
0.54824203
Computing Procrustes distance.
0.5348488092422485
Computing Projection distance.
0.45938748
Computing Spectral distance.
0.472787524716599
